<a href="https://colab.research.google.com/github/DIEGOCUM/SeminarioProfesional-Labs-16001350/blob/master/16001350_Lab3TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar TensorFlow, enable eager for tf iteration and import datasets 


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
# Installa TensorFlow

import tensorflow as tf
tf.enable_eager_execution()
tf.executing_eagerly()

!pip install -q tensorflow-datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tensorflow import keras
import numpy as np

print(tf.__version__)

#Usamos una version de un vocabularios precodificado de IMDB dataset y importamos el data set como tuplas 

In [0]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    as_supervised=True,
    with_info=True)

#Probamos codificar y decodificar una oracion y una parte del train data

In [3]:
encoder = info.features['text'].encoder

sample_string = 'Probamos el codificador de palabras.'

encoded_string = encoder.encode(sample_string)
print ('Codificamos el test {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print ('Mostramos la oracion original: "{}"'.format(original_string))


assert original_string == sample_string

for ts in encoded_string:
  print ('{} ----> {}'.format(ts, encoder.decode([ts])))

for train_example, train_label in train_data.take(1):
  print('Variable codificada  del train example:', train_example[90:100].numpy())
  print('Variable decodificada del train example:', encoder.decode(train_example[90:100].numpy()))
  print('Label:', train_label.numpy())



Codificamos el test [6691, 2827, 8, 1121, 4306, 6665, 5608, 7961, 510, 1920, 4870, 453, 7975]
Mostramos la oracion original: "Probamos el codificador de palabras."
6691 ----> Prob
2827 ----> amo
8 ----> s 
1121 ----> el 
4306 ----> cod
6665 ----> ifica
5608 ----> dor
7961 ---->  
510 ----> de 
1920 ----> pal
4870 ----> abr
453 ----> as
7975 ----> .
Variable codificada  del train example: [2836  311    5 5080 1209    3  183  117   35 1187]
Variable decodificada del train example: urist and cartoonist. He could be serious 
Label: 1


# Entrenando la data Data


In [0]:
BUFFER_SIZE = 1000

train_batches = (
    train_data
    .shuffle(BUFFER_SIZE)
    .padded_batch(32, train_data.output_shapes))

test_batches = (
    test_data
    .padded_batch(32, train_data.output_shapes))

# Construimos el Modelo y configuramos el optimizador y el loss function


In [6]:
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1, activation='sigmoid')])

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# usamos binary_crossentropy porque trabaja mejor probabilidades

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 130,977
Trainable params: 130,977
Non-trainable params: 0
_________________________________________________________________


# Entrenamos el modelo


In [7]:
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=test_batches,
                    validation_steps=30)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


782/782 [==============================] - 13s 17ms/step - loss: 0.6832 - acc: 0.6111 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.6265 - acc: 0.7494 - val_loss: 0.5991 - val_acc: 0.7833
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5487 - acc: 0.8005 - val_loss: 0.5340 - val_acc: 0.8146
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4820 - acc: 0.8354 - val_loss: 0.4808 - val_acc: 0.8417
Epoch 5/10
782/782 [==============================] - 10s 12ms/step - loss: 0.4285 - acc: 0.8592 - val_loss: 0.4385 - val_acc: 0.8490
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3866 - acc: 0.8730 - val_loss: 0.4057 - val_acc: 0.8594
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3538 - acc: 0.8838 - val_loss: 0.3802 - val_acc: 0.8646
Epoch 8/10
782/782 [==============================] - 10s 13ms/st

# Evaluamos el modelo

In [8]:
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

782/782 [==============================] - 7s 9ms/step - loss: 0.3346 - acc: 0.8749
Loss:  0.3346203866574313
Accuracy:  0.87488
